# Generative AI Custom Evaluation
This is an example notebook which showcases how a user can use AI Core custom evaluation to benchmark their large language models, evaluate orchestration configuration or prompts for their use case.
It uses publicly available [MedicationQA dataset](https://langtest.org/docs/pages/benchmarks/medical/medicationqa/) which consists of commonly asked consumer questions about medications. The workload computes industry standard metrics to check the reliability of the response generate by llm.
<br>**Note: For detailed instructions please refer to [Readme](./Readme.md)**

## Place your Dataset and Run configuration
Place your files as such:
1. Place your dataset in the folder `PUT_YOUR_DATASET_HERE`
2. Place your prompt template configurations in the folder `PUT_YOUR_PROMPT_TEMPLATE_HERE`
3. Place your custom metric files in folder `PUT_YOUR_CUSTOM_METRIC_HERE`


# SetUp (Step 1)


In [2]:
! pip install -r requirements.txt
! jupyter labextension install @jupyter-widgets/jupyterlab-manager

You should consider upgrading via the '/Users/c5408555/Downloads/notebook-update-v2/venv/bin/python3 -m pip install --upgrade pip' command.
(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
Building jupyterlab assets (production, minimized)


## Load your environment variables

Ensure that your environment variables are set in a `.env` file (see sample.env for an example). If there is a missing field the notebook will prompt you for a value.

In [1]:
# Loading the credentials from the env file
from gen_ai_hub.proxy.gen_ai_hub_proxy import GenAIHubProxyClient
from dotenv import load_dotenv
import os

load_dotenv(override=True)


# Fetching environment variables or prompting the user if missing
AICORE_BASE_URL = os.getenv("AICORE_BASE_URL") or input("AICORE_BASE_URL is missing. Please enter it: ")
AICORE_RESOURCE_GROUP = os.getenv("AICORE_RESOURCE_GROUP") or input("AICORE_RESOURCE_GROUP is missing. Please enter it (default: 'default'): ") or "default"
AICORE_AUTH_URL = os.getenv("AICORE_AUTH_URL") or input("AICORE_AUTH_URL is missing. Please enter it: ")
AICORE_CLIENT_ID = os.getenv("AICORE_CLIENT_ID") or input("AICORE_CLIENT_ID is missing. Please enter it: ")
AICORE_CLIENT_SECRET = os.getenv("AICORE_CLIENT_SECRET") or input("AICORE_CLIENT_SECRET is missing. Please enter it: ")

AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY") or input("AWS_ACCESS_KEY is missing. Please enter it: ")
AWS_BUCKET_ID = os.getenv("AWS_BUCKET_ID") or input("AWS_BUCKET_ID is missing. Please enter it: ")
AWS_REGION = os.getenv("AWS_REGION") or input("AWS_REGION is missing. Please enter it: ")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY") or input("AWS_SECRET_ACCESS_KEY is missing. Please enter it: ")
DEPLOYMENT_URL = os.getenv("DEPLOYMENT_URL", None)

# Initializing the GenAIHubProxyClient
client = GenAIHubProxyClient(
    base_url=AICORE_BASE_URL,
    auth_url=AICORE_AUTH_URL,
    client_id=AICORE_CLIENT_ID,
    client_secret=AICORE_CLIENT_SECRET,
    resource_group=AICORE_RESOURCE_GROUP
)

# Dependencies and Helper Functions (Step 2)

In [2]:
import os
import json

def get_prompt_config_file(folder_path):
    """
    Retrieves a list of all JSON file names in the specified folder.
    """
    if not os.path.isdir(folder_path):
        print(f"The folder path '{folder_path}' does not exist.")
        return []

    json_files = [file for file in os.listdir(folder_path) if file.endswith(".json")]

    if not json_files:
        print(f"No JSON files were found in the folder '{folder_path}'.")
    return json_files


def get_dataset_file_name(folder_path):
    """
    Retrieves the name of the first file in the specified folder.
    """
    if not os.path.isdir(folder_path):
        print(f"The folder path '{folder_path}' does not exist.")
        return None

    items_in_folder = os.listdir(folder_path)

    for item in items_in_folder:
        item_path = os.path.join(folder_path, item)
        if os.path.isfile(item_path):
            return item

    print(f"No files were found in the folder '{folder_path}'.")
    return None


def load_prompt_template(folder_path, file_name):
    """
    Loads the contents of a JSON prompt template into a variable.
    """
    file_path = os.path.join(folder_path, file_name)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)   # returns Python dict
    except Exception as e:
        print(f"Error loading prompt template: {e}")
        return None

# --- MAIN EXECUTION ---
PROMPT_FOLDER = "./PUT_YOUR_PROMPT_TEMPLATE_HERE"
DATASET_FOLDER = "./PUT_YOUR_DATASET_HERE"

PROMPT_CONFIG_FILES = get_prompt_config_file(PROMPT_FOLDER)
DATASET_NAME = get_dataset_file_name(DATASET_FOLDER)

if PROMPT_CONFIG_FILES and DATASET_NAME:
    # Load the first JSON prompt template
    PROMPT_TEMPLATE = load_prompt_template(PROMPT_FOLDER, PROMPT_CONFIG_FILES[0])
    print(f"Prompt configs: {PROMPT_CONFIG_FILES}")
    print(f"Dataset name: {DATASET_NAME}")
    print("Prompt template contents:", PROMPT_TEMPLATE)
else:
    print("Missing run or dataset file.")
    raise SystemExit("Exiting due to missing run/dataset file.")


Prompt configs: ['prompt_template.json']
Dataset name: medicalqna_dataset.csv
Prompt template contents: {'template': [{'role': 'user', 'content': 'List the benefits and side effects of the drug in the following consumer health question: {{?question}}.'}]}


### Register an Object Store Secret
To use the evaluations service, you must register an object store with the name default. Optionally, you can register an additional object store with a name of your choice.

In [3]:
# setup authentication and headers needed for AI Core requests
def _get_headers():
    headers = {
        "Authorization": client.get_ai_core_token(),
        "AI-Resource-Group": AICORE_RESOURCE_GROUP,
        "Content-Type": "application/json",
    }
    return headers

In [4]:
# Register S3 secret with AI Core which will be used an input source 
import requests
import json
import logging

def delete_oss_secret(oss_name=""):
    headers = _get_headers()
    
    DELETE_SECRETS_ENDPOINT = f'/v2/admin/objectStoreSecrets/{oss_name}'
    request_url = f"{AICORE_BASE_URL}{DELETE_SECRETS_ENDPOINT}"
    
    try:
        response = requests.delete(request_url, headers=headers, timeout=120)
        if response.status_code == 202:
            print(f"Successfully deleted object store secret: {oss_name}")
        elif response.status_code == 404:
            print(f"Object store secret not found: {oss_name}. It may not exist.")
        else:
            logging.error(f"Failed to delete object store secret: {oss_name}, Status Code: {response.status_code}")
    except Exception as e:
        logging.error(f"Error occurred while attempting to delete object store secret: {e}")
        raise

def register_oss_secret(oss_name="", path_prefix=""):
    headers = _get_headers()
    
    POST_SECRETS_ENDPOINT = '/v2/admin/objectStoreSecrets'
    request_url = f"{AICORE_BASE_URL}{POST_SECRETS_ENDPOINT}"
    
    request_body = {
        "name": oss_name,
        "data": {
            "AWS_ACCESS_KEY_ID": AWS_ACCESS_KEY,
            "AWS_SECRET_ACCESS_KEY": AWS_SECRET_ACCESS_KEY
        },
        "type": "S3",
        "bucket": AWS_BUCKET_ID,
        "endpoint": "s3-eu-central-1.amazonaws.com",
        "region": AWS_REGION,
        "pathPrefix": path_prefix,
        "verifyssl": "0",
        "usehttps": "1",
    }
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        result = response.json()
        return result
    except:
        logging.error("Error occurred while attempting to create object store secret")
        raise
        
delete_oss_secret(oss_name="default")
delete_oss_secret(oss_name="genai-quick-data-notebook")
        
register_oss_secret(oss_name="default", path_prefix="")
register_oss_secret(oss_name="genai-quick-data-notebook", path_prefix="")

Successfully deleted object store secret: default
Successfully deleted object store secret: genai-quick-data-notebook


{'message': 'secret has been created'}

In [5]:
# uploading these files to Object store to register as an artifact inside ai core

import boto3
import os
import uuid

def upload_folder_to_s3(folder_path, bucket_name, s3_prefix=""):
    """
    Upload a folder to an S3 bucket recursively.

    :param folder_path: The local folder path to upload.
    :param bucket_name: The name of the S3 bucket.
    :param s3_prefix: Optional prefix to use for the S3 keys (e.g., subfolder in the bucket).
    """
    s3_client = boto3.client(
            's3',
            aws_access_key_id=AWS_ACCESS_KEY,
            aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
            region_name=AWS_REGION
            )

    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            print("val of root is ", file_name)
            local_path = os.path.join(root, file_name)
            # Compute the relative path for the S3 key
            relative_path = os.path.relpath(local_path, folder_path)
            s3_key = os.path.join(s3_prefix, relative_path).replace("\\", "/")  # Ensure S3-compatible paths
            print("val of s3 key is ", s3_key)
            print(f"Uploading {local_path} to s3://{bucket_name}/{s3_key}")
            
            # Upload the file
            s3_client.upload_file(local_path, bucket_name, s3_key)

# Example usage
folder_to_upload_testdata = "./PUT_YOUR_DATASET_HERE"
folder_to_upload_custommetric = "./PUT_YOUR_CUSTOM_METRIC_HERE"
user_directory_prefix = "I321506" # replace with your i-number as string here
prefix_guid = user_directory_prefix if user_directory_prefix is not None else str(uuid.uuid4().hex)
s3_testdata_prefix = f"genaiEvaluation/{prefix_guid}/testdata" # Leave empty for root of the bucket


upload_folder_to_s3(folder_to_upload_testdata, AWS_BUCKET_ID, s3_testdata_prefix)
input_artifact_path = f"ai://genai-quick-data-notebook/genaiEvaluation/{prefix_guid}"

val of root is  medicalqna_dataset.csv
val of s3 key is  genaiEvaluation/I321506/testdata/medicalqna_dataset.csv
Uploading ./PUT_YOUR_DATASET_HERE\medicalqna_dataset.csv to s3://hcp-b60330de-a879-4848-9a3d-0ac828f4517c/genaiEvaluation/I321506/testdata/medicalqna_dataset.csv


The user stores the input files in the object store and registers the root folder as artifact with AI Core. The File Upload and Artifact endpoints of AI Core API may be used for this purpose. In this example `genaiEvaluation\{prefix_guid}` is the root folder containing the orchestration configurations and test data which is registered as AI Core artifact.

In [6]:
import requests
import logging
# Registering the uploaded files from AWS as artifacts to use inside configuration.

def register_artifact():
    headers = _get_headers()
    
    GET_ARTIFACTS_ENDPOINT = '/v2/lm/artifacts'
    request_url = f"{AICORE_BASE_URL}{GET_ARTIFACTS_ENDPOINT}"
    
    request_body = {
        "labels": [
            {
            "key": "ext.ai.sap.com/prompt-evaluation",
            "value": "true"
            }
        ],
        "name": "genai-eval-simplified-test-data",
        "kind": "other",
        "url": input_artifact_path, # input artifact path
        "description": "demo artifacts for evaluation flow.",
        "scenarioId": "genai-evaluations"
    }
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        result = response.json()
        print(result)
        return result['id']
    except:
        print("Error occurred while attempting to create an execution")
        raise
        

artifact_id = register_artifact()

{'id': '75dd115f-1707-4de6-9031-ce22d8c12015', 'message': 'Artifact acknowledged', 'url': 'ai://genai-quick-data-notebook/genaiEvaluation/I321506'}


## Create Orchestration Deployment
An orchestration Deployment URL is required for us to run our evaluation. Once created we need to wait until the deployment is running and provides us a deployment url which will be add to our configuration file in the next step. You can skip this step if you already have a orchestration deployment running.

In [7]:
import requests
import json
import time



def create_orchestration_configuration():
    headers = _get_headers()
    request_body = {
    "name": "orchestrationDeployment",
    "executableId": "orchestration",
    "scenarioId": "orchestration",
    "parameterBindings": [
        {
            "key": "modelFilterList",
            "value": "null"
        },
        {
            "key": "modelFilterListType",
            "value": "allow"
        }
    ],
    "inputArtifactBindings": []
    }
    
    GET_CONFIGURATIONS_ENDPOINT = '/v2/lm/configurations'
    request_url = f"{AICORE_BASE_URL}{GET_CONFIGURATIONS_ENDPOINT}"
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        print(response)
        if(response.status_code != 201):
            raise
        result = response.json()
        print(result)
        return result['id']
    except:
        logging.error("Error occurred while attempting to create a Configuration")
        raise
    
def execute_orchestration_deployment(configuration_id):
    headers = _get_headers()
    GET_DEPLOYMENTS_ENDPOINT = '/v2/lm/deployments'
    request_url = f"{AICORE_BASE_URL}{GET_DEPLOYMENTS_ENDPOINT}"
    
    request_body = {
        "configurationId": configuration_id
    }
    
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        print(response)
        if(response.status_code != 202):
            print("Deployment execution failed")
        result = response.json()
        print(result)
        return result['id']
    
    except:
        logging.error("Error occurred while attempting to create an execution")
        raise

def get_deployment_status(orchestration_deployment_id):
    headers = _get_headers()
    api_url = f"{AICORE_BASE_URL}/v2/lm/deployments/{orchestration_deployment_id}?$select=status"
    timeout = 400  
    initial_interval = 30  
    pending_interval = 10
    start = time.time()

    status = None
    current_interval = initial_interval

    while time.time() - start < timeout:
        response = requests.get(api_url, headers=headers)
        if response.status_code == 200:
            status = response.json().get('status')
            print(f"Deployment {orchestration_deployment_id} status: {status}")
            # Adjust polling interval based on status
            if status == 'RUNNING':
                return True
            elif status == 'UNKNOWN':
                current_interval = initial_interval
            elif status == 'PENDING':
                current_interval = pending_interval

        else:
            print(f"Failed to fetch deployment status. HTTP {response.status_code}")
            return False

        # Waiting according to status for API call
        time.sleep(current_interval)

def get_deployment_url(orchestration_deployment_id):
    headers = _get_headers()
    response = requests.get(f"{AICORE_BASE_URL}/v2/lm/deployments/{orchestration_deployment_id}", headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to get deployment URL: {response.status_code} - {response.text}")
    return response.json().get('deploymentUrl')

# You can skip this step if you already have a orchestration deployment running
deployment_url = DEPLOYMENT_URL
if not deployment_url:
    configuration_id = create_orchestration_configuration()
    orchestration_deployment_id = execute_orchestration_deployment(configuration_id)
    is_running = get_deployment_status(orchestration_deployment_id) 
    if is_running:
        deployment_url = get_deployment_url(orchestration_deployment_id)
        print(f"Deployment URL: {deployment_url}")
    else:
        print("Deployment is not running or failed.")

## Approach Selection

Select whether to use `Prompt Registry` or `Orchestration Registry` approach

In [7]:
from ipywidgets import Checkbox, VBox, HBox, Output, Label, Layout
from IPython.display import display
import textwrap

# --- Selection state ---
approach = None
suppress_update = False  

# --- Define options ---
flag_options = [
    "prompt_registry",
    "orchestration_registry"
]

# --- Output widget to show current selection ---

output = Output(layout=Layout(border="1px solid black", height="70px", overflow="auto", width="900px"))


# --- Handler for checkbox changes ---
def on_flag_change(change):
    global approach, suppress_update
    if suppress_update:
        return

    if change["new"]:  # A checkbox was checked
        suppress_update = True
        # Uncheck all other checkboxes
        for cb in checkboxes:
            if cb.description != change["owner"].description:
                cb.value = False
        suppress_update = False
        approach = change["owner"].description
    else:
        # Only clear if the unchecked one was the currently selected
        if approach == change["owner"].description:
            approach = None

    # Update display once per action
    with output:
        output.clear_output(wait=True)
        msg = f"Selected approach: {approach or 'None'}"
        wrapped = textwrap.fill(msg, width=60)
        output.append_stdout(wrapped + "\n")

# --- Create checkboxes ---
checkboxes = [
    Checkbox(value=False, description=option, layout=Layout(width="250px"))
    for option in flag_options
]

# --- Attach event handler ---
for cb in checkboxes:
    cb.observe(on_flag_change, names="value")

# --- Display UI ---
header = Label(
    value="Please select the configuration mode:",
    layout=Layout(margin="10px 0px 10px 0px")
)
ui = VBox([header, HBox(checkboxes), output])
display(ui)


In [8]:
# Select the approach, either "prompt_registry" or  orchestration_registry"
print(approach)

orchestration_registry


### Create a Prompt Template in Prompt Registry (Prompt Registry Approach)

The following code defines a function `create_prompt_template()` that creates a new **Prompt Template** in the SAP AI Core **Prompt Registry**.

**Note** : If you wish to use a prompt template that already exists in prompt registry, you can manually set `prompt_template_id` in the next cell and skip executing this cell

In [9]:
def create_prompt_template():
    headers = _get_headers()
    GET_PROMPT_TEMPLATES_ENDPOINT = '/v2/lm/promptTemplates'
    request_url = f"{AICORE_BASE_URL}{GET_PROMPT_TEMPLATES_ENDPOINT}"
    
    request_body = {
    "name": "prompt-registry-eval-acc-test",
    "version": "1.0.0",
    "scenario": "genai-evaluations",
    "spec": PROMPT_TEMPLATE
    }
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        if(response.status_code != 200):
            raise
        result = response.json()
        print(result)
        return result['id']
    except:
        logging.error("Error occurred while attempting to create a prompt template")
        raise

prompt_template_id = create_prompt_template()

{'message': 'Prompt updated successfully.', 'id': 'cf1908d7-c793-497d-ab8c-4e220f41102b', 'scenario': 'genai-evaluations', 'name': 'prompt-registry-eval-acc-test', 'version': '1.0.0'}


In [ ]:
# Manually set prompt_template_id here if you wish to use pre existing prompt template
# prompt_template_id=""

## Select your metrics
 
Tick the metrics you wish to use. If the widget does not load properly, you can manually fill in the string `selected_metrics_str`

**Note: If your dataset does not have a reference column, DO NOT Select metrcis where reference is required.**

In [10]:
from ipywidgets import Checkbox, VBox, HBox, Output, Label, Layout
import textwrap


# List to store selected values
selected_values = []

# Original list of checkbox options
checkbox_options = [
    "Content Filter on Input",
    "Content Filter on Output",
    "Pointwise Instruction Following",
    "Pointwise Answer Relevance",
    "Pointwise Conciseness"
]

# Additional options with "(reference required)" in their description
additional_options = [
    f"{option} (reference required)" for option in ["BLEU", "ROUGE", "BERT Score", "Pointwise Correctness"]
]

# Combine both lists
all_checkbox_options = checkbox_options + additional_options

# Function to handle checkbox changes
def on_checkbox_change(change):
    global selected_metrics_str  # Declare the string version of selected_values as global
    if change['new']:  # If the checkbox is checked
        # Remove "(reference required)" before adding to the list
        metric_name = change['owner'].description.replace(" (reference required)", "")
        if metric_name not in selected_values:
            selected_values.append(metric_name)
    else:  # If the checkbox is unchecked
        # Remove "(reference required)" before removing from the list
        metric_name = change['owner'].description.replace(" (reference required)", "")
        if metric_name in selected_values:
            selected_values.remove(metric_name)
    # Convert the list to a comma-separated string
    selected_metrics_str = ",".join(selected_values)
    # Display the updated list with wrapped text
    with output:
        output.clear_output(wait=True)  # Clear the output before printing
        wrapped_text = textwrap.fill(f"Selected values: {selected_values}", width=80)
        output.append_stdout(wrapped_text + "\n")  # Write directly to the output widget

# Create checkboxes for the combined options with wider layout
checkboxes = [
    Checkbox(value=False, description=option, layout=Layout(width="900px")) for option in all_checkbox_options
]

# Attach the change handler to each checkbox
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

# Group checkboxes into rows (2 per row)
rows = [HBox(checkboxes[i:i+2]) for i in range(0, len(checkboxes), 2)]

# Output widget to display the selected values with a scrollable and wrapped area
output = Output(layout=Layout(border="1px solid black", height="150px", overflow="auto", width="900px"))

# Header label
header = Label(value="Please choose which metrics you want to run", layout=Layout(margin="10px 0px 10px 0px"))

# Display the header, checkboxes in rows, and the output
display(VBox([header] + rows + [output]))

# Initialize the string version of selected_values
selected_metrics_str = ",".join(selected_values)


In [11]:
# Manual Selection of Metrics
#selected_metrics_str = "rouge"
print(selected_metrics_str)

BERT Score,Pointwise Conciseness


This script checks for an evaluation metric in SAP AI Core.

1. You can provide Metric ID's directly by setting the variable as comma separated string:
       user_metric_ids = `"<your_metric_id>"`
   - ✅ If the ID exists, it will be returned.
  
2. Script reads all `.json` and `.jsonl` files from `CUSTOM_METRIC_FOLDER` to load the custom metrics
   -The loaded metrics are stored in `custom_metric_list'
   - The script will use the contents of the `custom_metric_list`
     to search for an existing metric by scenario + name + version.

3. If no existing metric is found:
   - A new metric will be created using the details in `custom_metric_list`.
   - Required fields in custom_metric: scenario, name, version, evaluationMethod.

4. At the end:
   - The script prints the final Metric ID that was found or created.



In [12]:
import os
import json
import requests

# --- Load JSON / JSONL files ---
def load_all_metrics(folder_path):
    """
    Loads all JSON and JSONL files from a folder into a single list of dicts.
    """
    metrics = []
    files = [f for f in os.listdir(folder_path) if f.endswith((".json", ".jsonl"))]

    if not files:
        print(f"No JSON/JSONL files found in {folder_path}")
        return metrics

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read().strip()
                try:
                    data = json.loads(content)
                    if isinstance(data, list):
                        metrics.extend(data)
                    elif isinstance(data, dict):
                        metrics.append(data)
                except json.JSONDecodeError:
                    # Attempt to parse as JSONL line by line
                    for line in content.splitlines():
                        line = line.strip()
                        if not line:
                            continue
                        try:
                            metrics.append(json.loads(line))
                        except json.JSONDecodeError:
                            print(f"Skipping invalid JSON line in {file_name}: {line[:50]}...")
        except Exception as e:
            print(f"Error reading {file_name}: {e}")
    return metrics

# --- Fetch all metrics from SAP AI Core ---
def fetch_all_metrics():
    request_url = f"{AICORE_BASE_URL}/v2/lm/evaluationMetrics"
    resp = requests.get(request_url, headers=_get_headers())
    resp.raise_for_status()
    return resp.json().get("resources", [])

# --- Create or fetch a metric ---
def create_or_get_metric(custom_metric, user_metric_id=None):
    all_metrics = fetch_all_metrics()

    # 1️⃣ User-supplied ID lookup
    if user_metric_id:
        for m in all_metrics:
            if m.get("id") == user_metric_id:
                print(f"✅ Metric already exists by ID: {user_metric_id}")
                return user_metric_id
        print(f"⚠️ User metric ID {user_metric_id} not found, will only include if valid later")

    # 2️⃣ Check by scenario, name, version
    scenario = custom_metric.get("scenario")
    name = custom_metric.get("name")
    version = custom_metric.get("version")
    if not all([scenario, name, version]):
        raise ValueError("Metric must include 'scenario', 'name', and 'version'")

    for m in all_metrics:
        if (m.get("scenario") == scenario and
            m.get("name") == name and
            m.get("version") == version):
            metric_id = m.get("id")
            print(f"✅ Metric already exists: {scenario}/{name} v{version}, ID = {metric_id}")
            return metric_id

    # 3️⃣ Create metric if not found
    request_url = f"{AICORE_BASE_URL}/v2/lm/evaluationMetrics"
    required_fields = ["scenario", "name", "version", "evaluationMethod", "metricType"]
    for f in required_fields:
        if f not in custom_metric:
            raise ValueError(f"❌ Missing required field: {f}")

    resp = requests.post(request_url, headers=_get_headers(), json=custom_metric)
    resp.raise_for_status()
    metric_id = resp.json().get("id")
    print(f"✅ Metric created successfully: {name} v{version}, ID = {metric_id}")
    return metric_id

# --- Main pipeline ---
CUSTOM_METRIC_FOLDER = "./PUT_YOUR_CUSTOM_METRIC_HERE"
user_metric_ids = "d1868b00-1601-407a-92cd-0b9065682d1f,dbf56851-8444-45d3-a0c1-adbe210c7e771"  # set by user if needed

# 1️⃣ Load all metrics from JSON/JSONL
custom_metric_list = load_all_metrics(CUSTOM_METRIC_FOLDER)

# 2️⃣ Create/fetch metrics from SAP AI Core
metric_ids = []
for metric in custom_metric_list:
    try:
        metric_id = create_or_get_metric(metric)
        metric_ids.append(metric_id)
    except ValueError as e:
        print(f"Skipping metric due to error: {e}")

# 3️⃣ Validate user_metric_ids separately if provided
if user_metric_ids and user_metric_ids.strip():
    all_metrics = fetch_all_metrics()
    # Split comma-separated IDs and strip whitespace
    for uid in [uid.strip() for uid in user_metric_ids.split(",")]:
        if any(m.get("id") == uid for m in all_metrics):
            metric_ids.append(uid)
        else:
            print(f"⚠️ User metric ID {uid} does not exist in AI Core, skipping.")
# 4️⃣ Convert to comma-separated string
custom_metric_ids_str = ",".join(metric_ids)
print("✅ All processed metric IDs:", custom_metric_ids_str)


✅ Metric already exists: genai-evaluations/groundedness v0.0.1, ID = 2b3cc135-a031-4d93-8641-1f3833797034
✅ Metric already exists: genai-evaluations/groundedness v0.1.6, ID = 9b349f8e-cd39-486d-809c-3dcfa3b15ac7
⚠️ User metric ID d1868b00-1601-407a-92cd-0b9065682d1f does not exist in AI Core, skipping.
⚠️ User metric ID dbf56851-8444-45d3-a0c1-adbe210c7e771 does not exist in AI Core, skipping.
✅ All processed metric IDs: 2b3cc135-a031-4d93-8641-1f3833797034,9b349f8e-cd39-486d-809c-3dcfa3b15ac7


## Select your Models
 
Tick the metrics you wish to use. If the widget does not load properly, you can manually fill in the string `selected_models_str`



In [13]:
import requests
import textwrap
from functools import partial
from ipywidgets import Checkbox, VBox, HBox, Output, Label, Layout
from IPython.display import display


# --- Call the API ---
GET_MODELS_ENDPOINT = '/v2/lm/scenarios/foundation-models/models'
request_url = f"{AICORE_BASE_URL}{GET_MODELS_ENDPOINT}"
headers = _get_headers()  # your existing function
response = requests.get(request_url, headers=headers)
models_data = response.json()
# --- Extract model options ---
llm_options = []
resources = models_data.get("resources", [])
for m in resources:
    display_name = m.get("displayName") or m.get("name") or m.get("model", "Unknown")
    model_id = m.get("model", m.get("id", "unknown"))

    versions = m.get("versions", [])
    version = "latest"
    if versions:
        latest = next((v for v in versions if v.get("isLatest")), versions[0])
        version = latest.get("name") or "latest"

    label = f"{display_name} ({model_id}, v:{version})"
    value = f"{model_id}:{version}"
    llm_options.append({"label": label, "value": value})

# --- Selection state ---
selected_models = []
selected_models_str = ""  # <-- your comma-separated string
output = Output(layout=Layout(border="1px solid black", height="150px", overflow="auto", width="900px"))

def update_output():
    global selected_models_str
    selected_models_str = ",".join(selected_models)
    with output:
        output.clear_output(wait=True)
        if selected_models_str:
            wrapped_text = textwrap.fill(f"Selected models: {selected_models_str}", width=80)
            output.append_stdout(wrapped_text + "\n")

# --- Callback handler ---
def on_checkbox_change(model_value, change):
    # Ignore redundant triggers
    if change["old"] == change["new"]:
        return

    if approach == "prompt_registry":
        # Multiple selections allowed
        if change["new"]:
            if model_value not in selected_models:
                selected_models.append(model_value)
        else:
            if model_value in selected_models:
                selected_models.remove(model_value)
    else:
        # Single selection only
        if change["new"]:
            # Uncheck all other boxes
            for cb in checkboxes:
                if cb.model_value != model_value:
                    cb.unobserve_all()
                    cb.value = False
                    cb.observe(partial(on_checkbox_change, cb.model_value), names="value")
            selected_models.clear()
            selected_models.append(model_value)
        else:
            if model_value in selected_models:
                selected_models.remove(model_value)

    update_output()

# --- Create checkboxes + labels ---
checkboxes = []
checkbox_rows = []
for opt in llm_options:
    cb = Checkbox(value=False, indent=False, layout=Layout(width="30px"))
    cb.model_value = opt["value"]  # attach model identifier
    cb.observe(partial(on_checkbox_change, opt["value"]), names="value")
    lbl = Label(value=opt["label"], layout=Layout(width="850px"))
    checkboxes.append(cb)
    checkbox_rows.append(HBox([cb, lbl]))

# --- Display ---
header = Label(value="Please choose which LLM models you want to run:", layout=Layout(margin="10px 0px 10px 0px"))
display(VBox([header] + checkbox_rows + [output]))




In [14]:
# Manual selection of models
# selected_models_str="gpt-4o:2024-08-06"
print("Selected models string:", selected_models_str)


Selected models string: gpt-4o:2024-08-06


Create orchestration registry config (Only needed if you chose orchestraion registry approach)

In [15]:
def create_orchestration_registry_config():
    headers = _get_headers()
    CREATE_ORCHESTRATION_REGISTRY = '/v2/registry/v2/orchestrationConfigs'
    request_url = f"{AICORE_BASE_URL}{CREATE_ORCHESTRATION_REGISTRY}"
    model_name,model_version=selected_models_str.split(":")
    request_body = {
      "name": "genai-eval-test",
      "version": "1.0.0",
      "scenario": "genai-evaluations",
      "spec": {
        "modules": {
          "prompt_templating": {
            "model": {
              "name": model_name,
              "version": model_version
            },
            "prompt": PROMPT_TEMPLATE
          }
        }
      }
    }
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        if(response.status_code != 200):
            print(response.json())
            raise
        result = response.json()
        print(result)
        return result['id']
    except:
        logging.error("Error occurred while attempting to create a orchestration registry id")
        raise
orchestration_registry_id = create_orchestration_registry_config()

{'message': 'Orchestration config updated successfully.', 'id': '96f7424b-496d-4c47-8f14-0c6562e7744f', 'scenario': 'genai-evaluations', 'name': 'genai-eval-test', 'version': '1.0.0'}


# Start Evaluation Run (Step 3)

In [17]:

import json
test_data_path = f"testdata/{DATASET_NAME}" # specify the test data path here. For the full folder just specifying testdata will work
test_datasets = json.dumps({'path': test_data_path, 'type': 'csv'})
metrics_list = ",".join([selected_metrics_str,custom_metric_ids_str])
models_list = selected_models_str
print(f"Selected metrics: {metrics_list}")
print(f"Selected models: {models_list}")
#variable_mapping = json.dumps({'prompt/question': 'data/topic'}) # to map the question prompt variable to the entry in dataset.
# orchestration_deployment_url = deployment_url # needs to specify this to use a specific deployment id
orchestration_deployment_url = "https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d92895b42bdd6175"
repetitions = "1"

Selected metrics: BERT Score,Pointwise Conciseness,2b3cc135-a031-4d93-8641-1f3833797034,9b349f8e-cd39-486d-809c-3dcfa3b15ac7
Selected models: gpt-4o:2024-08-06


In [18]:
#  creating an AICORE Configuration.
import requests

request_body = {
    "name": "genai-eval-conf",
    "scenarioId": "genai-evaluations",
    "executableId": "genai-evaluations-simplified",
    "inputArtifactBindings": [
        {
            "key": "datasetFolder",
            "artifactId": artifact_id
        }
    ],
    "parameterBindings": [
        {
            "key": "repetitions",
            "value": repetitions
        },
        {
            "key": "orchestrationDeploymentURL",
            "value": orchestration_deployment_url
        },
        {
            "key": "metrics",
            "value": metrics_list
        },
        {
            "key": "testDataset",
            "value": test_datasets
        },
        {
            "key": "promptTemplate",
            "value": prompt_template_id if approach == "prompt_registry" else ""
        },
        {
            "key": "models",
            "value": models_list if approach == "prompt_registry" else ""
        },
        {
            "key": "orchestrationRegistryIds",
            "value": orchestration_registry_id if approach == "orchestration_registry" else ""
        }
    ]
}

def create_aicore_configuration():
    headers = _get_headers()
    GET_CONFIGURATIONS_ENDPOINT = '/v2/lm/configurations'
    request_url = f"{AICORE_BASE_URL}{GET_CONFIGURATIONS_ENDPOINT}"
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        print(response)
        if(response.status_code != 201):
            raise
        result = response.json()
        print(result)
        return result['id']
    except:
        logging.error("Error occurred while attempting to create a Configuration")
        raise
        
configuration_id = create_aicore_configuration()

<Response [201]>
{'id': 'fc21348f-4232-4579-8d21-062b88328ef0', 'message': 'Configuration created'}


## Evaluation Execution Creation
Once Configration is create, we create the AI Core execution which triggers the evaluation workload.


In [19]:
# create an execution with the created configuration.

import requests
def create_execution():
    headers = _get_headers()
    GET_EXECUTIONS_ENDPOINT = '/v2/lm/executions'
    request_url = f"{AICORE_BASE_URL}{GET_EXECUTIONS_ENDPOINT}"
    request_body = {"configurationId" : configuration_id} 
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(request_body), timeout=120
        )
        print("response received is ", response)
        result = response.json()
        print(result)
        return result['id']
    except:
        logging.error("Error occurred while attempting to create an execution")
        raise
 

execution_id = create_execution()

response received is  <Response [202]>
{'id': 'e8d0adb660289477', 'message': 'Execution scheduled', 'status': 'UNKNOWN', 'targetStatus': 'COMPLETED'}


In [20]:
# get execution status
import requests
def get_execution_status(execution_id):
    headers = _get_headers()
    LOG_EXECUTIONS_ENDPOINT = f'/v2/lm/executions/{execution_id}'
    request_url = f"{AICORE_BASE_URL}{LOG_EXECUTIONS_ENDPOINT}"
    try:
        response = requests.get(
            request_url, headers=headers, timeout=120
        )
        print("response received is ", response)
        result = response.json()
        return result
    except:
        logging.error("Error occurred while attempting to get execution status")
        raise
 

get_execution_status(execution_id)

response received is  <Response [200]>


{'id': 'e8d0adb660289477',
 'createdAt': '2025-11-14T05:36:34Z',
 'modifiedAt': '2025-11-14T05:36:34Z',
 'status': 'COMPLETED',
 'scenarioId': 'genai-evaluations',
 'configurationId': 'fc21348f-4232-4579-8d21-062b88328ef0',
 'targetStatus': 'COMPLETED',
 'submissionTime': '2025-11-14T05:36:56Z',
 'startTime': '2025-11-14T05:36:56Z',
 'completionTime': '2025-11-14T05:52:15Z',
 'configurationName': 'genai-eval-conf',
 'executableId': 'genai-evaluations-simplified',
 'outputArtifacts': [{'id': '80dfbccc-b559-496c-a03a-4d21b18c11be',
   'createdAt': '2025-11-14T05:52:06Z',
   'modifiedAt': '2025-11-14T05:52:06Z',
   'url': 'ai://default/e8d0adb660289477/evaluation_result',
   'name': 'evaluation_result',
   'kind': 'resultset',
   'description': '',
   'scenarioId': 'genai-evaluations',
   'executionId': 'e8d0adb660289477'},
  {'id': '2cb535ce-33d0-482d-a673-ba66b0e1df6f',
   'createdAt': '2025-11-14T05:50:23Z',
   'modifiedAt': '2025-11-14T05:50:23Z',
   'url': 'ai://default/e8d0adb660289

<b>

1.  Run the following cells only when the status field in the Execution response is "COMPLETED" to view the results.
2.  The status field progresses through different states over time: UNKNOWN → PENDING → RUNNING → COMPLETED. Ensure it reaches COMPLETED before proceeding.


Note: The targetStatus will always be COMPLETED from the start, as it represents the intended final state of the Execution. Do not confuse it with the actual status field.
</b>

# Evaluation Result (Step 4)
The evaluation job produces two outputs
1. A SQLite DB file which stores the orchestration input, orchestration output, values for all the metrics calculated for this orchestration output and statistics such as latency for this orchestration output. These metric values are called raw metric values. This SQLite DB file is stored in the object store as an AI Core output artifact.
2. A set of metrics whose values are aggregated from the raw metric values. The aggregate metrics are stored in the tracking service. The user-defined tags along with the run names are stored with the metrics.
Post execution completion user can see the runs generated by the workload along with the aggregate metrics by calling the tracking api as show below

In [21]:
# Get aggregate metrics using execution id
import requests
def retrieve_aggregate_metrics(execution_id):
    headers = _get_headers()
    GET_METRICS_ENDPOINT = f'/v2/lm/metrics?tagFilters=evaluation.ai.sap.com/child-of={execution_id}'
    request_url = f"{AICORE_BASE_URL}{GET_METRICS_ENDPOINT}"
    try:
        response = requests.get(request_url, headers=headers, timeout=120)
        print("response received is ", response)
        result = response.json()
        return result
    except:
        logging.error("Error occurred while attempting to retreive aggeregate metrics for the run")
        raise

runs_data = retrieve_aggregate_metrics(execution_id)


response received is  <Response [200]>


In [22]:
import pandas as pd
from IPython.display import HTML

def get_model_from_run(run):
    for tag in run.get("tags", []):
        if tag.get("name") == "evaluation.ai.sap.com/model":
            return tag.get("value")

def aggregate_metrics_by_model(runs_list):
    transformed_data = []
    for run in runs_list:
        model = get_model_from_run(run)
        for metric in run["metrics"]:
            output_json = {
                "model": model,
                "metrics_name": metric.get("name"),
                "metric_value": metric.get("value")
            }
            transformed_data.append(output_json)
    return transformed_data


def create_metrics_pivot_table(transformed_data):
    """
    Creates a pivot table where rows are models and columns are metrics.
    
    Args:
        transformed_data: List of dictionaries with 'model', 'metrics_name', 'metric_value'
    
    Returns:
        DataFrame with models as rows and metrics as columns
    """
    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(transformed_data)
    
    # Create pivot table
    pivot_table = df.pivot_table(
        index='model',
        columns='metrics_name',
        values='metric_value',
        aggfunc='first'  # Use 'first' to get the single value, or 'mean' if there are duplicates
    )
    
    return pivot_table

transformed_data = aggregate_metrics_by_model(runs_data['resources'])
metrics_pivot = create_metrics_pivot_table(transformed_data)

HTML(metrics_pivot.to_html())

metrics_name,BERT Score/F1/mean,BERT Score/F1/median,BERT Score/F1/p90,BERT Score/F1/p95,BERT Score/F1/stddev,BERT Score/Precision/mean,BERT Score/Precision/median,BERT Score/Precision/p90,BERT Score/Precision/p95,BERT Score/Precision/stddev,BERT Score/Recall/mean,BERT Score/Recall/median,BERT Score/Recall/p90,BERT Score/Recall/p95,BERT Score/Recall/stddev,Pointwise Conciseness/1/count,Pointwise Conciseness/2/count,Pointwise Conciseness/3/count,Pointwise Conciseness/4/count,Pointwise Conciseness/5/count,Pointwise Conciseness/mean,Pointwise Conciseness/median,Pointwise Conciseness/p90,Pointwise Conciseness/p95,Pointwise Conciseness/stddev,completion_tokens/sum,groundedness/0/count,groundedness/1/count,groundedness/2/count,groundedness/3/count,groundedness/4/count,groundedness/5/count,latency/average,prompt_tokens/sum,submission/sum
model,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
gpt-4o,0.466545,0.486934,0.550116,0.561422,0.077235,0.414502,0.445725,0.498497,0.505158,0.081079,0.539436,0.548189,0.63881,0.662922,0.081068,0.0,0.0,13.0,22.0,14.0,4.020408,4.0,5.0,5.0,0.749716,19553.0,0.0,0.0,0.0,0.0,0.0,0.0,105.284739,1608.0,49.0


## Process Your Execution Results
The next steps will take your evaluation run and process it such that it will give you a point based ranking of which particular model did the best based on the metrics you selected

In [23]:
import pandas as pd
from IPython.display import HTML

# Scoring logic depends on "scoring_type"
# "weight" represents the relative weight of this metric to all SELECTED metrics
METRICS_SCORING_TYPE_MAPPING = {
    "Content Filter on Input": {
        "scoring_type": "bool-false", # False is good
        "weight": 1
    },
    "Content Filter on Output": {
        "scoring_type": "bool-false", # False is good
        "weight": 1
    },
    "Pointwise Instruction Following": {
        "scoring_type": "num_1_to_5",
        "weight": 1
    },
    "Pointwise Answer Relevance": {
        "scoring_type": "num_1_to_5",
        "weight": 1
    },
    "Pointwise Conciseness": {
        "scoring_type": "num_1_to_5",
        "weight": 1
    },
    "Pointwise Correctness": {
        "scoring_type": "num_1_to_5",
        "weight": 1
    },
    "BLEU": {
        "scoring_type": "num_0_to_1",
        "weight": 1
    },
    "ROUGE": {
        "scoring_type": "num_0_to_1",
        "weight": 1
    },
    "BERT Score": {
        "scoring_type": "F1/Precision/Recall num_0_to_1",
        "weight": 1
    }
}

def calculate_bool_metric_score(pivot_df, metric_base_name, true_is_good):
    """
    Calculate scores for boolean metrics based on False/True counts.
    
    Args:
        pivot_df: DataFrame with models as rows and metrics as columns
        metric_base_name: Base name of the metric (without /False/count or /True/count)
        true_is_good: Boolean indicating if True is considered a good outcome
    
    Returns:
        Series with boolean metric scores per model (scaled to -1 to 1)
    """
    false_col = f"{metric_base_name}/False/count"
    true_col = f"{metric_base_name}/True/count"
    
    false_values = pivot_df[false_col] if false_col in pivot_df.columns else 0
    true_values = pivot_df[true_col] if true_col in pivot_df.columns else 0
    total_values = true_values + false_values

    score = ((false_values * 1) + (true_values * -1)) / total_values

    if true_is_good:
        score = 0 - score

    return score

def calculate_numeric_metric_score(pivot_df, metric_base_name, range_min=0, range_max=1):
    """
    Calculate scores for numeric metrics with /mean
    The mean is normalized to a score between -1 and 1 using the provided range.
    
    Args:
        pivot_df: DataFrame with models as rows and metrics as columns
        metric_base_name: Base name of the metric (without suffixes)
        range_min: Minimum possible value of the metric
        range_max: Maximum possible value of the metric
    
    Returns:
        Series with numeric metric scores per model (scaled to -1 to 1)
    """
    mean_col = f"{metric_base_name}/mean"
    
    if mean_col not in pivot_df.columns:
        return pd.Series(0.0, index=pivot_df.index)
    
    mean_values = pivot_df[mean_col]
    
    # Linear normalization from [range_min, range_max] to [0, 1]
    normalized = (mean_values - range_min) / (range_max - range_min)
    
    # Scale to [-1, 1]
    score = (normalized * 2) - 1
    
    return score

def calculate_bert_score(pivot_df, metric_base_name):
    """
    Calculate BERT Score by averaging F1, Precision, and Recall scores.
    
    Args:
        pivot_df: DataFrame with models as rows and metrics as columns
        metric_base_name: Base name "BERT Score"
    
    Returns:
        Series with BERT scores per model (scaled to -1 to 1)
    """
    f1_col = f"{metric_base_name}/F1/mean"
    precision_col = f"{metric_base_name}/Precision/mean"
    recall_col = f"{metric_base_name}/Recall/mean"
    
    scores = []
    for col in [f1_col, precision_col, recall_col]:
        if col in pivot_df.columns:
            scores.append(pivot_df[col])
    
    if not scores:
        return pd.Series(0.0, index=pivot_df.index)
    
    # Average the three metrics (already in 0 to 1 range)
    avg_score = sum(scores) / len(scores)
    
    # Scale to [-1, 1]
    score = (avg_score * 2) - 1
    
    return score

def find_unique_metrics_in_pivot(pivot_df):
    """
    Identify unique metric base names present in the pivot table.
    
    Args:
        pivot_df: DataFrame with models as rows and metrics as columns
    """
    # Extract unique metric names from pivot table columns
    unique_metrics = set()
    for col in pivot_df.columns:
        # Extract base metric name by removing suffixes
        base_name = col
        for suffix in ['/False/count', '/True/count', '/F1_score/mean','/Precision_score/mean', 
                       '/Recall_score/mean','/mean','/median', '/p90', '/p95', '/stddev']:
            if suffix in base_name and "BERT Score" not in base_name:
                base_name = base_name.replace(suffix, '')
                unique_metrics.add(base_name)
                break
        if base_name.startswith("BERT Score/"):
            base_name = "BERT Score"
            unique_metrics.add(base_name)
    if not unique_metrics:
        raise ValueError("No valid metrics found in pivot table")
    return unique_metrics


def rank_models(pivot_df, unique_metrics=None):
    """
    Rank models based on metrics present in the pivot table.
    
    Args:
        pivot_df: DataFrame with models as rows (index) and metrics as columns
    
    Returns:
        DataFrame with model rankings and scores
    """    
    # Calculate total weight for metrics present in pivot table
    total_weight = sum(METRICS_SCORING_TYPE_MAPPING[m]["weight"] for m in unique_metrics)
    
    # Initialize total score
    total_scores = pd.Series(0.0, index=pivot_df.index)
    
    # Process each metric found in the pivot table
    for metric_name in unique_metrics:
        config = METRICS_SCORING_TYPE_MAPPING[metric_name]
        scoring_type = config["scoring_type"]
        weight = config["weight"] / total_weight
        
        if scoring_type == "bool-false":
            # False is good (True is bad)
            metric_score = calculate_bool_metric_score(pivot_df, metric_name, true_is_good=False)
            total_scores += metric_score * weight
            
        elif scoring_type == "bool-true":
            # True is good (False is bad)
            metric_score = calculate_bool_metric_score(pivot_df, metric_name, true_is_good=True)
            total_scores += metric_score * weight
            
        elif scoring_type == "num_1_to_5":
            metric_score = calculate_numeric_metric_score(pivot_df, metric_name, range_min=1, range_max=5)
            total_scores += metric_score * weight
            
        elif scoring_type == "num_0_to_1":
            metric_score = calculate_numeric_metric_score(pivot_df, metric_name, range_min=0, range_max=1)
            total_scores += metric_score * weight
            
        elif scoring_type == "F1/Precision/Recall num_0_to_1":
            # BERT Score
            metric_score = calculate_bert_score(pivot_df, metric_name)
            total_scores += metric_score * weight
    
    # Create results DataFrame
    results_df = pd.DataFrame({
        'model': pivot_df.index,
        'total_score': total_scores.values
    })
    
    # Rank models (higher score = better rank)
    results_df['rank'] = results_df['total_score'].rank(ascending=False, method='min').astype(int)
    results_df = results_df.sort_values('rank')
    
    return results_df

def get_detailed_scores(pivot_df, unique_metrics):
    """
    Get detailed breakdown of scores per metric for each model.
    
    Args:
        pivot_df: DataFrame with models as rows and metrics as columns
    
    Returns:
        DataFrame with detailed scores per metric
    """
    detailed_scores = pd.DataFrame(index=pivot_df.index)
    
    # Process each metric in the mapping
    for metric_name in unique_metrics:
        scoring_type = METRICS_SCORING_TYPE_MAPPING[metric_name]["scoring_type"]
        
        if scoring_type == "bool-false":
            detailed_scores[f"{metric_name}_score"] = calculate_bool_metric_score(pivot_df, metric_name, true_is_good=False)
            
        elif scoring_type == "bool-true":
            detailed_scores[f"{metric_name}_score"] = calculate_bool_metric_score(pivot_df, metric_name, true_is_good=True)
            
        elif scoring_type == "num_1_to_5":
            detailed_scores[f"{metric_name}_score"] = calculate_numeric_metric_score(pivot_df, metric_name, range_min=1, range_max=5)
            
        elif scoring_type == "num_0_to_1":
            detailed_scores[f"{metric_name}_score"] = calculate_numeric_metric_score(pivot_df, metric_name, range_min=0, range_max=1)
            
        elif scoring_type == "F1/Precision/Recall num_0_to_1":
            detailed_scores[f"{metric_name}_score"] = calculate_bert_score(pivot_df, metric_name)
    
    return detailed_scores

unique_metrics = find_unique_metrics_in_pivot(metrics_pivot)

# Get detailed scores breakdown
detailed = get_detailed_scores(metrics_pivot, unique_metrics)
display(HTML(detailed.to_html()))

# Rank models
ranking = rank_models(metrics_pivot, unique_metrics)
display(HTML(ranking.to_html()))

,BERT Score_score,Pointwise Conciseness_score
model,,
gpt-4o,-0.053011,0.510204


,model,total_score,rank
0,gpt-4o,0.228596,1


To further drill down , User can also download the SQLite DB file from object storage and analyse the results(instance level metrics, logs etc) locally.

In [24]:
# download the result artifacts from Object store.
import boto3

def download_all_objects(prefix, destination_folder):
    """
    Recursively download all objects from an S3 bucket starting with a specific prefix.

    :param bucket_name: Name of the S3 bucket.
    :param prefix: Prefix to filter objects in the bucket.
    :param destination_folder: Local folder to save the downloaded files.
    """
    s3_client = boto3.client(
            's3',
            aws_access_key_id=AWS_ACCESS_KEY,
            aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
            region_name=AWS_REGION
            )

    # Ensure the destination folder exists
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Paginate through objects
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=AWS_BUCKET_ID, Prefix=prefix)

    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                key = obj['Key']
                local_file_path = os.path.join(destination_folder, os.path.relpath(key, prefix))

                # Ensure the local directory structure exists
                local_directory = os.path.dirname(local_file_path)
                if not os.path.exists(local_directory):
                    os.makedirs(local_directory)

                # Download the object
                print(f"Downloading {key} to {local_file_path}")
                s3_client.download_file(AWS_BUCKET_ID, key, local_file_path)


# Download the evaluation results from the object store. Look at execution status under "outputArtifacts" key to see the 'url'
# which shows the data path of where your output results are stored
EXECUTION_ID = execution_id
sqlite_db_prefix = f'{EXECUTION_ID}/evaluation_result/'  # change the prefix based on where your output artifact is stored in the bucket.
destination_folder = 'results-new'

download_all_objects(sqlite_db_prefix, destination_folder)

<b>NOTE: The below Cell shows results of top 10 rows of the Evaluation Results across all SQLite tables. IF you wish to see all the entries you can comment the line saying df.head(10) in the below cell or modify the number accordingly.</b>

In [25]:
# viewing the results from sqlite db in tabular format..
import sqlite3
import pandas as pd
from IPython.display import display, HTML

# Path to your SQLite database file
db_file = 'results-new/results.db'

connection = sqlite3.connect(db_file)

# Specify the table names you want to display
table_names = ['run','configuration', 'submission', 'submission_result', 'evaluation_result'] 

# Create the CSS and HTML container
html_content = """
<style>
/* Container to hold all tables */
.fixed-container {
    max-height: 600px;  /* Total container height */
    overflow-y: auto;   /* Vertical scroll for container */
    border: 2px solid #ddd;
    padding: 10px;
}

/* Table styling */
.table-container table {
    border-collapse: collapse; /* Merge table borders */
    width: 100%; /* Full width for tables */
    margin-bottom: 20px; /* Space between tables */
}

.table-container th, .table-container td {
    border: 1px solid #ddd; /* Add gridlines */
    text-align: left;       /* Align text to the left */
    padding: 8px;           /* Cell padding */
    white-space: nowrap;    /* Prevent text wrapping */
}

.table-container th {
    font-weight: bold;         /* Bold header text */
}

/* Allow dynamic column widths */
.table-container td {
    white-space: nowrap;   /* Prevent wrapping for long text */
}

</style>
<div class="fixed-container">
"""

for table_name in table_names:
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, connection)
    # If you want to see all the rows across all tables, remove/comment the next line
    df = df.head(10)  # Limiting the number of rows displayed
    table_html = df.to_html(classes='table-container', index=False)
    html_content += f"""
    <div class="table-container">
        <h3>Table: {table_name}</h3>
        {table_html}
    </div>
    """

html_content += "</div>"

display(HTML(html_content))

# Close the connection
connection.close()

In [94]:
#Delete Execution Id
def delete_execution():
    headers = _get_headers()
    EXEC_ID = execution_id
    GET_EXECUTIONS_ENDPOINT = '/v2/lm/executions/'
    request_url = f"{AICORE_BASE_URL}{GET_EXECUTIONS_ENDPOINT}{EXEC_ID}"
    try:
        response = requests.delete(
                request_url, headers=headers, params={"AI-Resource-Group":AICORE_RESOURCE_GROUP}, timeout=120
            )
        print(response)
        if(response.status_code != 202):
            raise
        result = response.json()
        print(result)
    except:
        logging.error("Error occurred while attempting to delete a Configuration")
        raise
    
delete_execution()

<Response [202]>
{'id': 'ef8963f68ec99242', 'message': 'Deletion scheduled', 'targetStatus': 'DELETED'}
